## Data Formatting and Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import helper_functions_elliot as hfe # Elliot's helper function
import windrose as wr # Creates plots of windroses
from scipy import stats
import math

# Read in all data files and rename attributes to similar format
boom1 = pd.read_csv("data/Boom1OneMin.csv")
boom1.rename(columns = {'TimeStamp':'time',
                        'MeanVelocity (m/s)':'meanVel6',
                       'MeanDirection':'meanDir6',
                       'MeanTemperature (C )':'meanTemp6',
                       'MeanPressure (mmHg)':'meanPres6'},inplace=True)
boom2 = pd.read_csv("data/Boom2OneMin-2.csv")
boom2.rename(columns = {'TIMESTAMP':'time',
                        'MeanVelocity (m/s)':'meanVel10',
                       'MeanDirection':'meanDir10',
                       'MeanTemperature (C )':'meanTemp10',
                       'MeanRH (%)':'meanRH10'},inplace=True)
boom3 = pd.read_csv("data/Boom3OneMin.csv")
boom3.rename(columns = {'TIMESTAMP':'time',
                        'MeanVelocity (m/s)':'meanVel20',
                       'MeanDirection':'meanDir20'},inplace=True)
boom4 = pd.read_csv("data/Boom4OneMin.csv")
boom4.rename(columns = {'TimeStamp':'time',
                        'MeanVelocity':'meanVel32',
                       'MeanDirection':'meanDir32',
                       'MeanTemperature':'meanTemp32',
                       'MeanRH':'meanRH32'},inplace=True)
boom5 = pd.read_csv("data/Boom5OneMin.csv")
boom5.rename(columns = {'TimeStamp':'time',
                        'MeanVelocity':'meanVel80',
                       'MeanDirection':'meanDir80',
                       'MeanTemperature':'meanTemp80',
                       'MeanRH':'meanRH80'},inplace=True)
boom6 = pd.read_csv("data/Boom6OneMin-2.csv")
boom6.rename(columns = {'TIMESTAMP':'time',
                        'MeanVelocity (m/s)':'meanVel106W',
                       'Mean Direction':'meanDir106W',
                       'MeanTemperature (C )':'meanTemp106W',
                       'MeanRH (%)':'meanRH106W'},inplace=True)
boom7 = pd.read_csv("data/Boom7OneMin.csv")
boom7.rename(columns = {'TimeStamp':'time',
                        'MeanVelocity (m/s)':'meanVel106E',
                       'MeanDirection':'meanDir106E',
                       'MeanPressure (mmHg)':'meanPres106E'},inplace=True)

boomList = [boom1,boom2,boom3,boom4,boom5,boom6,boom7]
for i in range(0,len(boomList)):
    boomList[i]['time'] = pd.to_datetime(boomList[i]['time']) # Format all date-times the same way

# Merge all dataframes to get all height-based attributes for each time
wind = pd.merge(boom1, boom2, how = 'inner', on = 'time')
wind = wind.merge(boom3, how = 'inner', on = 'time')
wind = wind.merge(boom4, how = 'inner', on = 'time')
wind = wind.merge(boom5, how = 'outer', on = 'time') # Outer join bc boom 5 is missing much data
wind = wind.merge(boom6, how = 'inner', on = 'time')
wind = wind.merge(boom7, how = 'inner', on = 'time')

heights = [6,10,20,32,80,'106E','106W']
for h in heights:
    wind[f'meanDir{h}'] = (wind[f'meanDir{h}'] - 90) % 360 # Correct the angles

wind['meanDir106'] = np.zeros(len(wind)) # Set only one direction for 106m
averageDir = (wind['meanDir106E'] + wind['meanDir106W']) / 2
for i in range(len(wind)):
    if averageDir[i] < 180:
        wind['meanDir106'][i] = wind['meanDir106E'][i] # Wind is coming from the east, use east boom
    else:
        wind['meanDir106'][i] = wind['meanDir106W'][i] # Wind is coming from the west, use west boom
wind.drop(columns = ['meanDir106E','meanDir106W'], inplace = True) # Drop useless vel/dir attributes

is260_280 = (wind['meanDir106'] < 280) & (wind['meanDir106'] > 260)
is70_110 = (wind['meanDir106'] < 110) & (wind['meanDir106'] > 70)
wind['meanVel106'] = np.zeros(len(wind)) # Set one velocity for 106m
for i in range(len(wind)): # For each time, find most accurate velocity by either averaging between east and west booms, or picking the windward side due to shadow effect in certain sectors
    if is260_280[i]:
        wind['meanVel106'][i] = wind['meanVel106W'][i]
    elif is70_110[i]:
        wind['meanVel106'][i] = wind['meanVel106E'][i]
    else:
        wind['meanVel106'][i] = np.mean([wind['meanVel106W'][i], wind['meanVel106E'][i]]) 
wind.head(10)

wind.set_index('time',inplace=True) # Reset the index to time

heights = [6,10,20,32,80,106] #this is if we wanted to average all height vectors
#heights = [10,106] # df with only relevant attributes
for h in heights:
    dirRad = np.deg2rad(wind[f'meanDir{h}']) # Convert degrees to radians
    wind[f'x_{h}'] = wind[f'meanVel{h}'] * np.sin(dirRad) # Get horizontal component of wind vector. Angle is from north.
    wind[f'y_{h}'] = wind[f'meanVel{h}'] * np.cos(dirRad) # Get vertical component of wind vector
np.sum(np.isnan(wind['x_106']))  
len(wind)

windSTD = wind.fillna(wind.mean()) # Fill nan with mean
windSTD = stats.zscore(windSTD) # Find z-scores
wind = wind[~(windSTD > 5).any(axis=1)] # Only keep rows whose attribute's standard deviations are less than 5 using boolean indexing
len(wind) # Dropped 1429 rows

# Take average of attributes for 10 consecutive minute data points. 
# Note that return is Nan if any data point in interval is nan or if a time in the interval is missing (non-consecutive) 
wind = wind.resample('10T').mean()
np.sum(np.isnan(wind['x_106']))

for h in heights:
    wind[f'meanVel{h}'] = np.sqrt(wind[f'x_{h}']**2 + wind[f'y_{h}']**2) # Find velocity of averaged vector
    wind[f'meanDir{h}'] = (np.rad2deg(np.arctan2(wind[f'x_{h}'], wind[f'y_{h}'])) + 360) % 360 # Use vector components to find angle
    wind.drop(columns = [f'x_{h}',f'y_{h}'], inplace = True) # Drop useless component attributes

# Drop nan data points
wind = wind.dropna(subset=['meanPres6', 'meanPres106E', 'meanTemp10', 'meanTemp106W', 'meanRH10', 'meanRH106W', 'meanVel10', 'meanDir10', 'meanVel106', 'meanDir106'])
np.sum(np.isnan(wind['meanVel106'])), len(wind)

C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\1891327497.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind['meanDir106'][i] = wind['meanDir106E'][i] # Wind is coming from the east, use east boom
C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\1891327497.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind['meanDir106'][i] = wind['meanDir106W'][i] # Wind is coming from the west, use west boom
C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\1891327497.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

(0, 28649)

In [ ]:
# Get richardson number for one data point
def BRN10(wind):
    P1 = wind['meanPres6'] # Pressure (mmHg) *Note were using pressure at boom 1 to approximate pressure at boom 2
    P2 = wind['meanPres106E']
    T1 = wind['meanTemp10'] + 273.15 # Temperature (K)
    T2 = wind['meanTemp106W'] + 273.15
    RH1 = wind['meanRH10'] # Relative humidity (%)
    RH2 = wind['meanRH106W']
    vpt1 = hfe.virtual_potential_temperature(RH1, P1, T1) # Get vpt
    vpt2 = hfe.virtual_potential_temperature(RH2, P2, T2)
    z1 = 6
    z2 = 106
    ws1 = wind['meanVel10'] # Wind speed at 10m
    wd1 = wind['meanDir10'] # Wind dir at 10m
    ws2 = wind['meanVel106'] # Wind speed at 106m
    wd2 = wind['meanDir106'] # Wind dir at 106m
    return hfe.bulk_richardson_number(vpt1, vpt2, z1, z2, ws1, ws2, wd1, wd2) # Get bulk richardson number (BRN)

# Get vector of BRN for entire dataset (remember its in 10min averages)
def BRN_vec(wind):
    length = len(wind)
    BRN = np.zeros(length) # Placeholder vector of zeros

    for i in range(length):
        wind_10 = wind.iloc[i] # Select data point
        BRN[i] = BRN10(wind_10) # Fill vector with BRN

    return BRN

BRN = BRN_vec(wind) # Utilize function
wind['BRN'] = BRN # Create attribute for the BRN

C:\Users\Hudson\GLWind\helper_functions_elliot.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  ri = g * delta_vpt * delta_z / (vpt_avg * (delta_u * delta_u + delta_v * delta_v))


In [ ]:
# ln(U(z)) = ln(beta) + alpha*ln(z)
def alpha_est(z,U):
    #alpha = np.sum(np.log(z) - np.mean(np.log(z)) * (np.log(U) - np.mean(np.log(U)))) / np.sum((np.log(z) - np.mean(np.log(z)))**2)
    alpha = (len(z) * np.sum(np.log(U)*np.log(z)) - np.sum(np.log(z)) * np.sum(np.log(U))) / (len(z)*np.sum(np.log(z)**2) - np.sum(np.log(z))**2)
    if len(z) == 0:
        print(z,U,len(z)*np.sum(np.log(z)**2),np.sum(np.log(z))**2)
    return alpha

def beta_est(z,U):
    alpha_b = alpha_est(z,U)
    beta = np.mean(np.log(U)) - alpha_b * np.mean(np.log(z))
    return np.exp(beta)

def alpha_vector(wind):
    length = len(wind)
    alpha_vec = np.zeros(length)
    for i in range(length):
        z = np.array([6,10,20,32,106]) # No 80m because of missing data
        U = np.array([wind['meanVel6'].iloc[i], wind['meanVel10'].iloc[i], wind['meanVel20'].iloc[i], wind['meanVel32'].iloc[i], wind['meanVel106'].iloc[i]])
        alpha_vec[i] = alpha_est(z,U)
    return alpha_vec

def beta_vector(wind):
    length = len(wind)
    beta_vec = np.zeros(length)
    for i in range(length):
        z = np.array([6,10,20,32,106])
        U = np.array([wind['meanVel6'].iloc[i], wind['meanVel10'].iloc[i], wind['meanVel20'].iloc[i], wind['meanVel32'].iloc[i], wind['meanVel106'].iloc[i]])
        beta_vec[i] = beta_est(z,U)
    return beta_vec

wind['alpha'] = alpha_vector(wind)

C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\3622547192.py:4: RuntimeWarning: divide by zero encountered in log
  alpha = (len(z) * np.sum(np.log(U)*np.log(z)) - np.sum(np.log(z)) * np.sum(np.log(U))) / (len(z)*np.sum(np.log(z)**2) - np.sum(np.log(z))**2)
C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\3622547192.py:4: RuntimeWarning: invalid value encountered in double_scalars
  alpha = (len(z) * np.sum(np.log(U)*np.log(z)) - np.sum(np.log(z)) * np.sum(np.log(U))) / (len(z)*np.sum(np.log(z)**2) - np.sum(np.log(z))**2)


In [42]:
windt = wind.query('(300 < meanDir10 < 330) | (120 < meanDir10 < 150)') # filter for only rough and smooth terrain
windt['terrain'] = np.zeros(len(windt)) # fill a new terrain column with zeros
for i in range(len(windt)):
    if (300 < windt['meanDir10'][i] < 330):
        windt['terrain'].iloc[i] = 1 # terrain is rough
    else:
        windt['terrain'].iloc[i] = 0 # terrain is smooth
windt = windt[['meanVel10','meanVel106','BRN','terrain','alpha']] # choose attributes of interest
windt = windt.dropna()
windt.head() # length 6362

C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\3544284911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windt['terrain'] = np.zeros(len(windt)) # fill a new terrain column with zeros
C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\3544284911.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  windt['terrain'].iloc[i] = 0 # terrain is smooth
C:\Users\Hudson\AppData\Local\Temp\ipykernel_14528\3544284911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

,meanVel10,meanVel106,BRN,terrain,alpha
time,,,,,
2017-09-22 20:00:00,2.247963,7.484448,0.274555,0.0,0.508970
2017-09-22 20:10:00,2.525433,8.023636,0.240095,0.0,0.470577
2017-09-22 20:20:00,2.182813,8.045008,0.248538,0.0,0.531764
2017-09-22 20:30:00,2.346829,8.950391,0.182004,0.0,0.534520
2017-09-22 21:20:00,2.130514,8.967270,0.185975,0.0,0.567082


In [43]:
windt_temp = windt # define new copy of df
windt_temp = windt_temp.sample(frac=1).reset_index(drop=True) # shuffle rows, reset index
wind_data_training = windt_temp.iloc[:5000,:] # first randomized 5000 rows
wind_data_test = windt_temp.iloc[5000:,:] # all the rows after

In [44]:
wind_data_training.to_csv('wind_data_training.csv', index=False)
wind_data_test.to_csv('wind_data_test.csv', index=False)